In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
#Install the required packages for this project
!pip install einops datasets bitsandbytes accelerate peft flash_attn
# !pip uninstall -y transformers
# !pip install git+https://github.com/huggingface/transformers
# !pip install --upgrade torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 85.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.1

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig

model_name = "microsoft/phi-2"

# Configuration to load model in 4-bit quantized
bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type='nf4',
                                bnb_4bit_compute_dtype='float16',
                                #bnb_4bit_compute_dtype=torch.bfloat16,
                                bnb_4bit_use_double_quant=True)


#Loading Microsoft's Phi-2 model with compatible settings
model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto',
                                             quantization_config=bnb_config,
                                             #attn_implementation="flash_attention_2",# to increase the training speed
                                             trust_remote_code=True)

# Setting up the tokenizer for Phi-2
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          add_eos_token=True,
                                          trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
!pip install py2neo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 4.5 MB/s eta 0:00:00


In [6]:
from py2neo import Graph, Node, Relationship
graph = Graph("bolt+s://d0642942.databases.neo4j.io:7687", auth=("neo4j", "AHHzMMC8_dusNm8T6bdkgs0lClYkdUg3VvChFi8d5GI"))

In [7]:
%cd /content/drive/MyDrive/ontology/phi2/

/content/drive/MyDrive/ontology/phi2


In [8]:
def get_related_question(question):
    query = """
    CALL db.index.fulltext.queryNodes("questionIndex",$question)
    YIELD node, score
    RETURN node.name, score
    ORDER BY score DESC
    """
    result = graph.run(query, question=question).data()
    return result[0]

def get_related_data(action):
    query = """
    MATCH (a:Action {name: $action})
    OPTIONAL MATCH (a)-[:HAS_STEP]->(s:Step)
    OPTIONAL MATCH (a)-[:HAS_REQUIREMENT]->(r:Requirement)
    OPTIONAL MATCH (a)-[:HAS_KEY]->(k:Keys)
    WITH a, collect(DISTINCT s.name) AS steps, collect(DISTINCT r.name) AS requirements, collect(DISTINCT k.name) AS keys
    RETURN steps, requirements, keys
    """
    result = graph.run(query, action=action).data()
    return result[0]

def get_action(question):
    query = """
    MATCH (q:Question {name: $question})<-[:HAS_QUESTION]-(a:Action)
    RETURN a.name AS action
    """
    result = graph.run(query, question=question).data()
    return result[0]


def get_context(q_txt):
    related_data = get_related_question(q_txt)

    action=get_action(related_data['node.name'])

    related_data = get_related_data(action['action'])

    key_txt="#Key words:"
    keys=related_data['keys']
    for k in keys:
        key_txt=key_txt+k+','

    req_txt="#Prerequisites:"
    requirements=related_data['requirements']
    for r in requirements:
        req_txt=req_txt+r+','

    step_txt="#Steps to Follow:"
    steps=related_data['steps']
    for s in steps:
        step_txt=step_txt+s+','

    context=key_txt+'\n'+req_txt+'\n'+step_txt
    return context

In [9]:
#Function that creates a prompt from instruction, context, category and response and tokenizes it
def collate_and_tokenize(q_text):
    if q_text:
      context=get_context(q_text)
    else:
      context="No Context"

    #Merging into one prompt for tokenization and training
    prompt = f"""###System:Read the Context and Chat History provided and answer the corresponding question.
    ###Context:
    {context}
    ###Question:
    {q_text}
    ###Answer:"""

    inputs = tokenizer(prompt, return_tensors="pt",
                   return_attention_mask=False,
                   padding=True, truncation=True)
    inputs.to('cuda')
    return inputs

Next, lets run the model with lora config and check inference on it.

In [10]:
from peft import PeftModel, PeftConfig

#Load the model weights from hub
model_id = "result/"
trained_model = PeftModel.from_pretrained(model, model_id)

#

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


ValueError: You are trying to offload the whole model to the disk. Please use the `disk_offload` function instead.

In [ ]:
def spliit_and_return(txt):
  parts=txt.split("###Answer:")
  return parts[0],parts[1]

In [ ]:
def get_answer_from_ontology(question):
  inputs=collate_and_tokenize(question)
  outputs = trained_model.generate(**inputs, max_length=2048)
  out_put_text = tokenizer.batch_decode(outputs,skip_special_tokens=True)[0]
  qustion_txt, prediction = spliit_and_return(out_put_text)
  return prediction

In [ ]:
q="how to create a account"
answer=get_answer_from_ontology(q)
print(answer)